# Top 5 YTD % Change on Metals Commodities


In [ ]:
import tradingeconomics as te
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [ ]:
te.login()

In [ ]:
data = te.getMarketsData(marketsField = 'commodities', output_type = 'df')
list_of_commodities = data['Group'].unique()
list_of_commodities

In [ ]:
def top_5_ytd_percentual_change(commodities_group):   
    ytd = data.query(f'Group == "{commodities_group}"').sort_values(by='YTDPercentualChange', ascending=False)
    ytd = ytd.head().reset_index(drop=True)
    ytd = ytd[['Name','Last','Group','YTDPercentualChange']]
    return ytd

In [ ]:
top_5_ytd_percentual_change(list_of_commodities[5])


### Top 5 Lithium importers

##### Getting a list of all countries available

In [ ]:
all_countries_available = te.getCmtCountry(output_type='df')


In [ ]:
all_countries_available_treated = all_countries_available[['name','iso']]
all_countries_available_treated = all_countries_available_treated.drop([0,288,289])

all_countries_available_list = all_countries_available_treated['name'].to_list()
all_countries_available_list[0:5]

##### We have to get a commodity category which have some kind of correlation with lithium. We suspect that primary batteries and primary cells could help us 

In [ ]:
te.getCmtCategories(output_type='df').query('name == "Primary cells & batteries, parts"')

#####  We have to check all countries primary cell and primary batteries, parts to get the main countries traders 

##### to get a country commodity group

In [ ]:
def get_country_commodity(country,category,type,year):
    country_with_no_data_list = []
#     print(f'getting {country}')
    country_category = te.getCmtCountryByCategory(country=country,type=type,category=category, output_type='df')
#     print(country_category)
    if country_category is None:
        country_with_no_data_list.append(country)
#         print(country_with_no_data_list)
    else:
        country_category = country_category.query(f'date == {year}')
        country_category = country_category[['country1','value']]
        return country_category.groupby(["country1"]).sum()


##### To get a dataframe with all countries and a specific commodity

In [ ]:
def dataframe_countries_commodities(countries_list,category,type,year):

    main_dataframe = pd.DataFrame()
    for country in countries_list:
    #     print(f'passing {country}')
    #     main_dataframe.append(get_country_commodity(country=country,category='Primary Cells and Primary Batteries',type='import',year='2020' ), ignore_index=True)
#         pre_dataframe = get_country_commodity(country=country,category='Primary Cells and Primary Batteries',type='import',year='2020')
        pre_dataframe = get_country_commodity(country=country,category=category,type=type,year=year)
        
        main_dataframe = main_dataframe.append(pre_dataframe)
    return main_dataframe.sort_values('value', ascending=False)

#### Top 5 - Importers

In [ ]:
lithium_top5_importers = dataframe_countries_commodities(countries_list=all_countries_available_list,category='Primary Cells and Primary Batteries', type='import',year='2020')
# lithium_top5_importers[0:5]

In [ ]:
x_import = lithium_top5_importers.index[0:5]
y_import = lithium_top5_importers['value'][0:5]
# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(
            x=x_import, y=y_import,
            text=y_import,
            textposition='auto',
        )])
fig.show(renderer='plotly_mimetype')

### Top 5 Exporters

In [ ]:
lithium_top5_exporters = dataframe_countries_commodities(countries_list=all_countries_available_list,category='Primary Cells and Primary Batteries', type='export',year='2020')
# lithium_top5_exporters[0:5]

In [ ]:
x_export = lithium_top5_exporters.index[0:5]
y_export = lithium_top5_exporters['value'][0:5]

# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(
            x=x_export, y=y_export,
            text=y_export,
            textposition='auto',
        )])

fig.show(renderer='plotly_mimetype')

In [ ]:
# tennessee.matos@tradingeconomics.com 